<a href="https://colab.research.google.com/github/BatoolMM/fastai/blob/master/Fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The First Chapter**

In [3]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [4]:
from fastbook import *
from fastai.vision.all import *
# !pip install ipywidgets
# import ipywidgets as widgets
# !jupyter nbextension enable --py widgetsnbextension
# !python3 -m pip install --upgrade pip
# !python3 -m pip install --upgrade Pillow

In [5]:
data = untar_data(URLs.PETS)/'images'

In [6]:
def is_cat(x):
    return x[0].isupper()

In [7]:
data_str = ImageDataLoaders.from_name_func(data, 
                                           get_image_files(data),
                                           valid_pct = 0.2, 
                                           seed =42,
                                           label_func = is_cat,
                                           item_tfms = Resize(224))

In [8]:
model = cnn_learner(data_str, resnet34, metrics=error_rate)
model.fine_tune(1)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,error_rate,time
0,0.161170,0.025869,0.008119,00:51


epoch,train_loss,valid_loss,error_rate,time
0,0.057979,0.026976,0.008119,00:55


In [9]:
image_uploader = widgets.FileUpload()
image_uploader

FileUpload(value={}, description='Upload')

In [10]:
img = PILImage.create(image_uploader.data[0])

In [ ]:
is_cat, _,prob = model.predict(img)

In [12]:
print(f"Is this a cat?: {is_cat}.")
print(f"Prob it's a cat: {prob[1].item():6f}")

Is this a cat?: True.
Prob it's a cat: 1.000000


# **The Second Chapter**

In [15]:
Normal_range = range(60, 120)
glucose = int(input('what is your glucose reading?:'))

if glucose in Normal_range:
  print('Normal')
elif glucose < 60:
  print('Low')
else:
  print('High')

what is your glucose reading?:30
low


In [20]:
normal_range = range(60,120)
glucose = int(input("what is your glucose reading? "))

if glucose in normal_range:
   print("Your glucose level is in the normal range")
else:
   print("Your glucose level i is not the normal range")
  

what is your glucose reading? 60
Your glucose level {} is in the normal range


In [21]:
 glucose = int(input("what is your glucose reading? "))

 if glucose in range(60,120):
   print(f"Your glucose level {glucose} is in the normal range 60-120")
else:
   print(f"Your glucose level {glucose} is not the normal range 60-120")

what is your glucose reading? 50
Your glucose level 50 is not the normal range 60-120
